In [3]:
import torch
import torch.nn as nn
import pandas as pd

In [4]:
from isaac.dataset import read_dataset, prepare_dataset
from isaac.constants import POSITION_COLS, MASS_CLASS_COLS, BASIC_TRAINING_COLS, FORCE_CLASS_COLS, PUCK_SQUARE_DISTANCES, PUCK_ANGLE_FEATURES
from isaac.sanity import class_proportions
from isaac.models import initialise_model
from isaac.training import training_loop
import matplotlib.pyplot as plt
import numpy as np

In [5]:
from isaac.utils import get_cuda_device_if_available
device = get_cuda_device_if_available()
print(device)

cuda:0


In [6]:
NORMALISE_DATA = True
BATCH_SIZE = 128
EPOCHS = 25
STEP_SIZE = 1
SEQ_END = 1800

In [7]:
TR_COLS = BASIC_TRAINING_COLS

In [8]:
train_trials = read_dataset("data/train_passive_trials.h5", n_trials=3500, cols=TR_COLS)
val_trials = read_dataset("data/val_passive_trials.h5", n_trials=500, cols=TR_COLS)

100%|██████████| 10/10 [00:00<00:00, 123.00it/s]


# MASS TRAINING

In [9]:
INPUT_DIM = len(TR_COLS)    # input dimension
HIDDEN_DIM = 25  # hidden layer dimension
N_LAYERS = 4     # number of hidden layers
OUTPUT_DIM = 3   # output dimension
DROPOUT = 0.5
network_params = (INPUT_DIM, HIDDEN_DIM, N_LAYERS, OUTPUT_DIM, DROPOUT)

In [10]:
labels = ["1800", "1500", "1200", "900", "600", "300"]
stats_dfs = []

loaders, scaler = prepare_dataset([train_trials, val_trials], class_columns=MASS_CLASS_COLS, 
                                  training_columns=TR_COLS, batch_size=BATCH_SIZE, 
                                  normalise_data=NORMALISE_DATA, device=device)

for seq_length, label in zip([1800, 1500, 1200, 900, 600, 300], labels):
    seq_start = SEQ_END - seq_length
    
    for seed in [0, 42, 72]:
        df = pd.DataFrame(columns=["seq_length", "Epoch", "Loss"])


        model, error, optimizer = initialise_model(network_params, lr=0.01, seed=seed, device=device)
        epoch_losses, epoch_accuracies, best_model = training_loop(model, optimizer, error, loaders[0], 
                                                                   loaders[1], EPOCHS, seq_start=seq_start, 
                                                                   seq_end=SEQ_END, step_size=STEP_SIZE)
                
        df["Epoch"] = np.arange(EPOCHS)
        df["Loss"] = epoch_losses
        df["Train Accuracy"] = epoch_accuracies[0]
        df["Val Accuracy"] = epoch_accuracies[1]
        df["seq_length"] = label
        stats_dfs.append(df)
        
stats = pd.concat(stats_dfs)
stats.to_hdf("sequence_length_plots/mass_stats.h5", key="stats")

100%|██████████| 10/10 [00:00<00:00, 1019.27it/s]
Train_loss (1.11)	 Train_acc (70.00)	 Val_acc (40.00): 100%|██████████| 2/2 [00:01<00:00,  1.51it/s] 
Train_loss (1.16)	 Train_acc (60.00)	 Val_acc (30.00): 100%|██████████| 2/2 [00:01<00:00,  1.54it/s] 
Train_loss (1.15)	 Train_acc (60.00)	 Val_acc (30.00): 100%|██████████| 2/2 [00:01<00:00,  1.52it/s] 
Train_loss (1.10)	 Train_acc (70.00)	 Val_acc (40.00): 100%|██████████| 2/2 [00:01<00:00,  1.82it/s] 
Train_loss (1.15)	 Train_acc (70.00)	 Val_acc (20.00): 100%|██████████| 2/2 [00:01<00:00,  1.84it/s] 
Train_loss (1.16)	 Train_acc (60.00)	 Val_acc (40.00): 100%|██████████| 2/2 [00:01<00:00,  1.83it/s] 
Train_loss (1.13)	 Train_acc (60.00)	 Val_acc (40.00): 100%|██████████| 2/2 [00:00<00:00,  2.30it/s] 
Train_loss (1.16)	 Train_acc (60.00)	 Val_acc (30.00): 100%|██████████| 2/2 [00:00<00:00,  2.33it/s] 
Train_loss (1.15)	 Train_acc (60.00)	 Val_acc (40.00): 100%|██████████| 2/2 [00:00<00:00,  2.20it/s] 
Train_loss (1.10)	 Train_acc (70

# FORCE TRAINING

In [11]:
labels = ["1800", "1500", "1200", "900", "600", "300"]
stats_dfs = []

loaders, scaler = prepare_dataset([train_trials, val_trials], class_columns=FORCE_CLASS_COLS, 
                                  training_columns=TR_COLS, batch_size=BATCH_SIZE, 
                                  normalise_data=NORMALISE_DATA, device=device)

for seq_length, label in zip([1800, 1500, 1200, 900, 600, 300], labels):
    seq_start = SEQ_END - seq_length
    
    for seed in [0, 42, 72]:
        df = pd.DataFrame(columns=["seq_length", "Epoch", "Loss"])


        model, error, optimizer = initialise_model(network_params, lr=0.01, seed=seed, device=device)
        epoch_losses, epoch_accuracies, best_model = training_loop(model, optimizer, error, loaders[0], 
                                                                   loaders[1], EPOCHS, seq_start=seq_start, 
                                                                   seq_end=SEQ_END, step_size=STEP_SIZE)
                
        df["Epoch"] = np.arange(EPOCHS)
        df["Loss"] = epoch_losses
        df["Train Accuracy"] = epoch_accuracies[0]
        df["Val Accuracy"] = epoch_accuracies[1]
        df["seq_length"] = label
        stats_dfs.append(df)
        
stats = pd.concat(stats_dfs)
stats.to_hdf("sequence_length_plots/force_stats.h5", key="stats")

100%|██████████| 10/10 [00:00<00:00, 1085.54it/s]
Train_loss (1.08)	 Train_acc (60.00)	 Val_acc (30.00): 100%|██████████| 2/2 [00:01<00:00,  1.52it/s] 
Train_loss (1.14)	 Train_acc (60.00)	 Val_acc (20.00): 100%|██████████| 2/2 [00:01<00:00,  1.55it/s] 
Train_loss (1.09)	 Train_acc (60.00)	 Val_acc (30.00): 100%|██████████| 2/2 [00:01<00:00,  1.59it/s] 
Train_loss (1.06)	 Train_acc (40.00)	 Val_acc (30.00): 100%|██████████| 2/2 [00:01<00:00,  1.78it/s] 
Train_loss (1.17)	 Train_acc (50.00)	 Val_acc (40.00): 100%|██████████| 2/2 [00:01<00:00,  1.84it/s] 
Train_loss (1.12)	 Train_acc (70.00)	 Val_acc (30.00): 100%|██████████| 2/2 [00:01<00:00,  1.86it/s] 
Train_loss (1.09)	 Train_acc (40.00)	 Val_acc (30.00): 100%|██████████| 2/2 [00:00<00:00,  2.34it/s] 
Train_loss (1.14)	 Train_acc (70.00)	 Val_acc (20.00): 100%|██████████| 2/2 [00:00<00:00,  2.34it/s] 
Train_loss (1.12)	 Train_acc (60.00)	 Val_acc (30.00): 100%|██████████| 2/2 [00:00<00:00,  2.32it/s] 
Train_loss (1.08)	 Train_acc (60